In [10]:
import pandas as pd
import numpy as np
data = pd.read_excel("dosie.xls").as_matrix()

good = []
bad = []

for d in data:
    d = d[0].lower()
    if ('(внесен'.decode(encoding='utf8') in d) and ('досье на проект'.decode(encoding='utf8') in d):
        if '(подписан'.decode(encoding='utf8') in d:
            good.append(d[16:d.rindex('(')])
        if '(снят'.decode(encoding='utf8') in d:
            bad.append(d[16:d.rindex('(')])

authors = []
names = []

all_authors = ['ВАС'.decode(encoding='utf8'), 'ГД'.decode(encoding='utf8'), 'СФ'.decode(encoding='utf8'), 'верховный суд'.decode(encoding='utf8'), 'местный орган'.decode(encoding='utf8'), 'правительство'.decode(encoding='utf8'), 'президент'.decode(encoding='utf8'), 'другое'.decode(encoding='utf8')]
    
for d in good:
    author = d[d.rindex('(внесен'.decode(encoding='utf8'))+8:d.rindex(')')]
    if ('республики'.decode(encoding='utf8') in author) or ('обл.'.decode(encoding='utf8') in author) or ('мосгор'.decode(encoding='utf8') in author) or ('окружн'.decode(encoding='utf8') in author) or ('город'.decode(encoding='utf8') in author) or ('севастополя'.decode(encoding='utf8') in author) or ('санкт-петербурга'.decode(encoding='utf8') in author) or ('округа'.decode(encoding='utf8') in author) or ('областн'.decode(encoding='utf8') in author) or ('области'.decode(encoding='utf8') in author) or ('губернской'.decode(encoding='utf8') in author) or ('краев'.decode(encoding='utf8') in author) or ('края'.decode(encoding='utf8') in author):
        author = 'местный орган'.decode(encoding='utf8')
    if ('гд'.decode(encoding='utf8') in author) or ('верховным советом'.decode(encoding='utf8') in author) or ('деп.'.decode(encoding='utf8') in author) or ('депутат'.decode(encoding='utf8') in author):
        author = 'ГД'.decode(encoding='utf8')
    if ('сф'.decode(encoding='utf8') in author) or ('советом федерации'.decode(encoding='utf8') in author) or ('совета федерации'.decode(encoding='utf8') in author):
        author = 'СФ'.decode(encoding='utf8')
    if ('правительство'.decode(encoding='utf8') in author) or ('првительство'.decode(encoding='utf8') in author) :
        author = 'правительство'.decode(encoding='utf8')
    if ('президент'.decode(encoding='utf8') in author) :
        author = 'президент'.decode(encoding='utf8')
    if ('арбитражным'.decode(encoding='utf8') in author) :
        author = 'ВАС'.decode(encoding='utf8')
    if ('верховным судом'.decode(encoding='utf8') in author) or ('верховного суда'.decode(encoding='utf8') in author) or ('вс '.decode(encoding='utf8') in author) :
        author = 'верховный суд'.decode(encoding='utf8')
    if author not in all_authors:
        author = 'другое'.decode(encoding='utf8')
    authors.append(all_authors.index(author))
    name = d[:d.rindex('(внесен'.decode(encoding='utf8'))]
    names.append(name)

    
for d in bad:
    
    if len(names) >= len(good)*2:
        break
    
    author = d[d.rindex('(внесен'.decode(encoding='utf8'))+8:d.rindex(')')]
    if ('республики'.decode(encoding='utf8') in author) or ('обл.'.decode(encoding='utf8') in author) or ('мосгор'.decode(encoding='utf8') in author) or ('окружн'.decode(encoding='utf8') in author) or ('город'.decode(encoding='utf8') in author) or ('севастополя'.decode(encoding='utf8') in author) or ('санкт-петербурга'.decode(encoding='utf8') in author) or ('округа'.decode(encoding='utf8') in author) or ('областн'.decode(encoding='utf8') in author) or ('области'.decode(encoding='utf8') in author) or ('губернской'.decode(encoding='utf8') in author) or ('краев'.decode(encoding='utf8') in author) or ('края'.decode(encoding='utf8') in author):
        author = 'местный орган'.decode(encoding='utf8')
    if ('гд'.decode(encoding='utf8') in author) or ('верховным советом'.decode(encoding='utf8') in author) or ('деп.'.decode(encoding='utf8') in author) or ('депутат'.decode(encoding='utf8') in author):
        author = 'ГД'.decode(encoding='utf8')
    if ('сф'.decode(encoding='utf8') in author) or ('советом федерации'.decode(encoding='utf8') in author) or ('совета федерации'.decode(encoding='utf8') in author):
        author = 'СФ'.decode(encoding='utf8')
    if ('правительство'.decode(encoding='utf8') in author) or ('првительство'.decode(encoding='utf8') in author) :
        author = 'правительство'.decode(encoding='utf8')
    if ('президент'.decode(encoding='utf8') in author) :
        author = 'президент'.decode(encoding='utf8')
    if ('арбитражным'.decode(encoding='utf8') in author) :
        author = 'ВАС'.decode(encoding='utf8')
    if ('верховным судом'.decode(encoding='utf8') in author) or ('верховного суда'.decode(encoding='utf8') in author) or ('вс '.decode(encoding='utf8') in author) :
        author = 'верховный суд'.decode(encoding='utf8')
    if author not in all_authors:
        author = 'другое'.decode(encoding='utf8')
    authors.append(all_authors.index(author))
    name = d[:d.rindex('(внесен'.decode(encoding='utf8'))]
    names.append(name)
    
from nltk.tokenize import RegexpTokenizer
from collections import Counter,defaultdict
tokenizer = RegexpTokenizer(r"\w+")

#словарь для всех токенов
token_counts = Counter()

#все заголовки и описания
all_texts = names


#считаем частоты слов
for s in all_texts:
    tokens = tokenizer.tokenize(s)
    for token in tokens:
        token_counts[token] +=1
        
min_count = 10
tokens = list(filter(lambda t: token_counts[t]>10, token_counts ))

token_to_id = {t:i+1 for i,t in enumerate(tokens)}
null_token = "NULL"
token_to_id[null_token] = 0

def vectorize(strings, token_to_id, max_len=150):
    token_matrix = []
    for s in strings:
        #if type(s) is not str:
        #    token_matrix.append([0]*max_len)
        #    continue
        s = s.lower()
        tokens = tokenizer.tokenize(s)
        token_ids = list(map(lambda token: token_to_id.get(token,0), tokens))[:max_len]
        token_ids += [0]*(max_len - len(token_ids))
        token_matrix.append(token_ids)

    return np.array(token_matrix)

len_of_names = []
for n in names:
    len_of_names.append(len(n))
    
names_tokens = vectorize(names,token_to_id,max_len=300)

target = list(np.ones(len(good)))
target.extend(list(np.zeros(len(good))))

from sklearn.cross_validation import train_test_split
data_tuple = train_test_split(names_tokens, authors, target)

train_names, test_names, train_authors, test_authors, train_target, test_target = data_tuple

train_names = np.array(train_names)
train_authors = np.array(train_authors)
trainX = np.hstack((train_names, train_authors[:,None]))
trainY = train_target
testX = np.hstack((test_names, np.array(test_authors)[:,None]))
testY = test_target
all_X = np.vstack((trainX, testX))
all_Y = np.vstack((np.array(trainY)[:,None], np.array(testY)[:,None]))

from sklearn.ensemble import GradientBoostingClassifier as Cl
classif = Cl()
classif.fit(all_X, all_Y)

import telebot
from telebot import types
token = '393691033:AAE4BKLNYu8ektUcHB6pvkzHwTrEepKKGiY'

def make_prediction(name, author):
    re_name = vectorize(np.array([name]),token_to_id,max_len=300)
    #print(re_name)
    #print(re_name.shape)
    #print(np.array([author]).shape)
    thisX = np.hstack((re_name, np.array([author])[:,None]))
    #print(thisX)
    return classif.predict_proba(thisX)[0][1]

T_name = 'Ф'
T_author = -1

bot = telebot.TeleBot(token)

@bot.message_handler(content_types=["text"])
def do_it(m):
    global T_name
    global T_author
    
    if m.text == '/help':
        Help(m)
    if m.text == '/start':
        start(m)
    if m.text == '/test_bill':
        test_bill(m)
    if 'Название'.decode(encoding='utf8') in m.text:
        T_name = m.text[8:]
        get_author(m)
        
    if m.text == 'Высший Арбитражный Суд'.decode(encoding='utf8'):
        T_author = 0   
        some_text(m, T_name, 'Высший Арбитражный Суд')
    if m.text == 'Государственная Дума'.decode(encoding='utf8'):
        T_author = 1
        some_text(m, T_name, 'Государственная Дума')
    if m.text == 'Совет Федерации'.decode(encoding='utf8'):
        T_author = 2
        some_text(m, T_name, 'Совет Федерации')
    if m.text == 'Верховный суд'.decode(encoding='utf8'):
        T_author = 3
        some_text(m, T_name, 'Верховный суд')
    if m.text == 'местный орган (субъект РФ)'.decode(encoding='utf8'):
        T_author = 4
        some_text(m, T_name, 'местный орган (субъект РФ)')
    if m.text == 'Правительство РФ'.decode(encoding='utf8'):
        T_author = 5
        some_text(m, T_name, 'Правительство РФ')
    if m.text == 'Президент РФ'.decode(encoding='utf8'):
        T_author = 6
        some_text(m, T_name, 'Президент РФ')
    if m.text == 'другое'.decode(encoding='utf8'):
        T_author = 7
        some_text(m, T_name, 'другое')
        
    if m.text == '/analyze':
        pr = make_prediction(T_name, T_author)
        s = 'Я считаю, данный законопроект будет принят с вероятностью' + str(int(pr*100)) + '%.\n\nНапоминаю, что точность моего анализа 85%, то есть я не могу гарантировать правильное определение исхода в абсолютно всех случаях.'
        bot.send_message(m.chat.id, s)
        T_name = ''
        T_author = -1
    
        
def some_text(m, name, author):
        s = 'Здорово! Проверьте, правильно ли вы всё ввели.\n\n'
        bot.send_message(m.chat.id, s)
        bot.send_message(m.chat.id, name)
        s = 'Автор документа: '
        s += author
        bot.send_message(m.chat.id, s)
        s =  'Если вы ошиблись, начните заново - отправьте /test_bill\n'
        s += 'Если всё верно, я готов анализировать данные. Отправьте /analyze'
        bot.send_message(m.chat.id, s)
        
def start(m):
    s = 'Привет, я Билл!\nЯ умею определять с точностью более 85% будет ли законопроект принят по его названию и автору.\n'
    s += '\nСоветую использовать настоящие названия законопроектов. Названия типа "какая-то фигня" дадут нерелевантный результат.\n'
    s += 'Хотите попробовать? Отправьте /test_bill\n\nЕсли хотите узнать обо мне больше, отправьте /help'
    bot.send_message(m.chat.id, s)
def test_bill(m):
    s = 'Хорошо, введите слово "Название", а затем название законопроекта, который я буду анализировать.\nДа, слово "Название" обязательно, извините.'
    bot.send_message(m.chat.id, s)
    
def get_author(m):
    keyboard = types.ReplyKeyboardMarkup(resize_keyboard=True)
    keyboard.add(*[types.KeyboardButton(choice) for choice in 
['Высший Арбитражный Суд', 'Государственная Дума', 'Совет Федерации', 'Верховный суд',
 'местный орган (субъект РФ)', 'Правительство РФ', 'Президент РФ', 'другое']])
    bot.send_message(m.chat.id, 'Отлично. Теперь выберите автора законопроекта.', reply_markup=keyboard)

def Help(m):
    s = 'Меня зовут Билл, и я умею определять с точностью более 85% будет ли законопроект принят по его названию и автору.\n'
    s += 'Я создан Екатериной Шестаковой (e-mail: oskolok-mirosdanija@yandex.ru) 09.05.2017.\n'
    s += 'Я создан при помощи машинного обучения на языке программирования Python.\n\n'
    s += 'Если вы хотите опробовать мою работу, отправьте /test_bill'
    s += '\nСоветую использовать настоящие названия законопроектов. Названия типа "какая-то фигня" дадут нерелевантный результат.'
    bot.send_message(m.chat.id, s)


In [19]:
if __name__ == '__main__':
    bot.polling(none_stop=True)

ConnectionError: HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot393691033:AAE4BKLNYu8ektUcHB6pvkzHwTrEepKKGiY/getUpdates?timeout=20&offset=1 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7f1377a00410>: Failed to establish a new connection: [Errno -2] Name or service not known',))